In [3]:
import numpy as np
import pandas as pd
import torch as torch

In [4]:
time_varying = np.random.randn(100, 10, 450)
static = np.random.randn(100, 10)

In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, random_split
from torch.utils.data.distributed import DistributedSampler

In [194]:
class Encoder(nn.Module):
    
    def __init__(self, ks = 2, pad = 1):
        
        super(Encoder, self).__init__()
        
        self.static_layer = nn.Linear(10, 50)
        
        self.conv_layer = nn.Sequential(
            nn.Conv1d(10, 25, kernel_size = ks, padding = pad, dilation = 1),
            nn.MaxPool1d(2),
            nn.ReLU(inplace = True),
            nn.Conv1d(25, 25, kernel_size = ks, padding = pad, dilation  = 2),
            nn.MaxPool1d(2),
            nn.ReLU(inplace = True),
            nn.Conv1d(25, 50, kernel_size = ks, padding = pad, dilation = 7),
            nn.MaxPool1d(2),
            nn.ReLU(inplace = True),
            nn.Flatten())
        
    def forward(self, static, time_varying):
        
        static_enc = self.static_layer(static)
        
        time_varying_enc = self.conv_layer(time_varying)
        
        return torch.cat((static_enc, time_varying_enc), axis = 1)
    

In [195]:
class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.2):
        super().__init__()
        
        self.out1 = nn.Linear(input_size, hidden_size)
        self.out2 = nn.Linear(hidden_size, output_size)
        self.attention = False
        self.dropout = nn.Dropout(dropout)

    def forward(self, encoding):
        hidden = self.out1(encoding)
        output = self.out2(hidden)
        return output

In [83]:
time_varying[1][:, :385].shape

(10, 385)

In [221]:
class poorMansMQCNN(nn.Module):
    
    def __init__(self, encoder, decoder, num_timesteps, output_size, hidden_size, input_size):
        
        super().__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_timesteps = num_timesteps
        
        self.encoder = Encoder()
        self.decoder = Decoder(input_size, hidden_size, output_size)
        
    def forward(self, static, time_varying):
        
        encoding = self.encoder(static, time_varying[:, :self.num_timesteps])
        
        output = self.decoder(encoding)
        
        return output

In [222]:
p = poorMansMQCNN(e, d, num_timesteps = 365, 
                  output_size = 85, 
                  hidden_size = 500, 
                  input_size = 2700)

In [223]:
s = torch.tensor(static).float()
t = torch.tensor(time_varying).float()

In [211]:
t[:, :365].shape

torch.Size([10, 365])

In [225]:
p(s, t).shape

torch.Size([100, 85])